### Code generation with RAG and self-correction
![image](../image/2_chatbots/Code_Assisant/main.png)

In [1]:
import os
os.chdir("../../")
os.getcwd()

'/home/yeonwoo/code/study'

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/concepts/#langchain-expression-language-lcel"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

### OpenAI

# Grader prompt
prompt = """당신은 LangChain 표현언어인 LCEL에 대한 전문성을 갖춘 코딩 어시스턴트입니다.\n
다음은 전체 LCEL 문서 세트입니다: \n ------- \n {context} \n ------- \n 사용자에게 답변
위에 제공된 문서를 기반으로 한 질문입니다. 제공한 코드가 실행될 수 있는지 확인하십시오. \n
필요한 모든 가져오기 및 변수가 정의되어 있습니다. 코드 솔루션에 대한 설명을 사용하여 답변을 구성하세요. \n
그런 다음 수입품을 나열하십시오. 마지막으로 작동하는 코드 블록을 나열합니다. 사용자 질문은 다음과 같습니다.
"""
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"{prompt}",
        ),
        ("placeholder", "{messages}"),
    ]
)


# Data model
class code(BaseModel):
    """Schema for code solutions to questions about LCEL."""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")


llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
code_gen_chain_oai = code_gen_prompt | llm.with_structured_output(code)
question = "How do I build a RAG chain in LCEL? please say korean"
solution = code_gen_chain_oai.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)
solution

code(prefix='이 문서에서는 LCEL을 사용하여 RAG(Retrieval Augmented Generation) 체인을 구축하는 방법을 설명합니다. RAG 체인은 외부 지식 베이스를 사용하여 언어 모델의 성능을 향상시키는 기술입니다. 이를 위해서는 벡터 저장소와 검색기를 설정하고, 이를 통해 검색된 정보를 기반으로 언어 모델이 응답을 생성하도록 구성해야 합니다.', imports="import { VectorStore, Retriever, LLM, RAGChain } from 'langchain';", code="// 벡터 저장소를 초기화합니다.\nconst vectorStore = new VectorStore({\n  // 벡터 저장소 설정\n});\n\n// 검색기를 초기화합니다.\nconst retriever = new Retriever({\n  vectorStore,\n  // 검색기 설정\n});\n\n// 언어 모델을 초기화합니다.\nconst llm = new LLM({\n  // 언어 모델 설정\n});\n\n// RAG 체인을 구성합니다.\nconst ragChain = new RAGChain({\n  retriever,\n  llm,\n  // 추가 설정\n});\n\n// RAG 체인을 실행하여 결과를 얻습니다.\nconst result = ragChain.run({\n  query: '질문을 입력하세요',\n});\n\nconsole.log(result);")

In [7]:
from langchain_core.prompts import ChatPromptTemplate

### Anthropic

# Prompt to enforce tool use
code_gen_prompt_claude = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """<instructions> You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user  question based on the \n 
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined. Structure your answer: 1) a prefix describing the code solution, 2) the imports, 3) the functioning code block. \n
    Invoke the code tool to structure the output correctly. </instructions> \n Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)

llm = ChatOpenAI(
    model="gpt-4o-2024-08-06", temperature=0,
    default_headers={"anthropic-beta": "tools-2024-04-04"},
)
structured_llm_claude = llm.with_structured_output(code, include_raw=True)

# Optional: Check for errors in case tool use is flaky
def check_claude_output(tool_output):
    """Check for parse error or failure to call the tool"""

    # Error with parsing
    if tool_output["parsing_error"]:
        # Report back output and parsing errors
        print("Parsing error!")
        raw_output = str(tool_output["raw"].content)
        error = tool_output["parsing_error"]
        raise ValueError(
            f"Error parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
        )

    # Tool was not invoked
    elif not tool_output["parsed"]:
        print("Failed to invoke tool!")
        raise ValueError(
            "You did not use the provided tool! Be sure to invoke the tool to structure the output."
        )
    return tool_output


# Chain with output check
code_chain_claude_raw = (
    code_gen_prompt_claude | structured_llm_claude | check_claude_output
)


def insert_errors(inputs):
    """Insert errors for tool parsing in the messages"""

    # Get errors
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        (
            "assistant",
            f"Retry. You are required to fix the parsing errors: {error} \n\n You must invoke the provided tool.",
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }


# This will be run as a fallback chain
fallback_chain = insert_errors | code_chain_claude_raw
N = 3  # Max re-tries
code_gen_chain_re_try = code_chain_claude_raw.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)


def parse_output(solution):
    """When we add 'include_raw=True' to structured output,
    it will return a dict w 'raw', 'parsed', 'parsing_error'."""

    return solution["parsed"]


# Optional: With re-try to correct for failure to invoke tool
code_gen_chain = code_gen_chain_re_try | parse_output

# No re-try
code_gen_chain = code_gen_prompt_claude | structured_llm_claude | parse_output

In [8]:
# Test
question = "How do I build a RAG chain in LCEL? please say korean"
solution = code_gen_chain.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)
solution

code(prefix='이 코드는 LCEL에서 RAG 체인을 구축하는 방법을 보여줍니다. Retrieval Augmented Generation (RAG)은 외부 지식 베이스와 결합하여 언어 모델을 향상시키는 기술입니다.', imports="import { RAGChain } from 'langchain';\nimport { VectorStore } from 'langchain/vectorstores';\nimport { Retriever } from 'langchain/retrievers';", code="// 벡터 스토어와 리트리버를 설정합니다.\nconst vectorStore = new VectorStore({\n  // 벡터 스토어 설정\n});\n\nconst retriever = new Retriever({\n  vectorStore: vectorStore,\n  // 리트리버 설정\n});\n\n// RAG 체인을 생성합니다.\nconst ragChain = new RAGChain({\n  retriever: retriever,\n  // 추가 설정\n});\n\n// RAG 체인을 사용하여 쿼리를 실행합니다.\nconst query = '질문을 입력하세요';\nconst result = await ragChain.run(query);\nconsole.log(result);")

In [9]:
from typing import List
from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error: str
    messages: List
    generation: str
    iterations: int

In [10]:
### Parameter

# Max tries
max_iterations = 3
# Reflect
# flag = 'reflect'
flag = "do not reflect"

### Nodes


def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [
            (
                "user",
                "Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:",
            )
        ]

    # Solution
    code_solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}",
        )
    ]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {
        "generation": code_solution,
        "messages": messages,
        "iterations": iterations,
        "error": "no",
    }


def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]

    # Prompt reflection

    # Add reflection
    reflections = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [("assistant", f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


### Edges


def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"

In [11]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code", code_check)  # check code
workflow.add_node("reflect", reflect)  # reflect

# Build graph
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "generate")
app = workflow.compile()

In [12]:
question = "How can I directly pass a string to a runnable and use it to construct the input needed for my prompt? prease say korean"
solution = app.invoke({"messages": [("user", question)], "iterations": 0, "error": ""})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: FINISH---


In [13]:
solution["generation"]

code(prefix='이 코드는 문자열을 직접 Runnable에 전달하고 이를 사용하여 프롬프트에 필요한 입력을 구성하는 방법을 보여줍니다.', imports='from langchain_core.prompts import PromptTemplate\nfrom langchain_core.runnables import Runnable', code='# Define a simple prompt template\nprompt_template = PromptTemplate(template="Translate the following text to English: {text}")\n\n# Create a Runnable with the prompt template\ndef create_runnable(input_text):\n    # Construct the input for the prompt\n    prompt_input = {\'text\': input_text}\n    \n    # Create a Runnable instance\n    runnable = Runnable(prompt_template, prompt_input)\n    \n    # Execute the Runnable\n    result = runnable.invoke()\n    return result\n\n# Example usage\ninput_string = "안녕하세요"\noutput = create_runnable(input_string)\nprint(output)')

In [14]:
import langsmith

client = langsmith.Client()

In [15]:
# Clone the dataset to your tenant to use it
try:
    public_dataset = (
        "https://smith.langchain.com/public/326674a6-62bd-462d-88ae-eea49d503f9d/d"
    )
    client.clone_public_dataset(public_dataset)
except:
    print("Please setup LangSmith")

In [16]:
from langsmith.schemas import Example, Run


def check_import(run: Run, example: Example) -> dict:
    imports = run.outputs.get("imports")
    try:
        exec(imports)
        return {"key": "import_check", "score": 1}
    except Exception:
        return {"key": "import_check", "score": 0}


def check_execution(run: Run, example: Example) -> dict:
    imports = run.outputs.get("imports")
    code = run.outputs.get("code")
    try:
        exec(imports + "\n" + code)
        return {"key": "code_execution_check", "score": 1}
    except Exception:
        return {"key": "code_execution_check", "score": 0}

In [17]:
def predict_base_case(example: dict):
    """Context stuffing"""
    solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": [("user", example["question"])]}
    )
    return {"imports": solution.imports, "code": solution.code}


def predict_langgraph(example: dict):
    """LangGraph"""
    graph = app.invoke(
        {"messages": [("user", example["question"])], "iterations": 0, "error": ""}
    )
    solution = graph["generation"]
    return {"imports": solution.imports, "code": solution.code}

In [18]:
from langsmith.evaluation import evaluate

# Evaluator
code_evalulator = [check_import, check_execution]

# Dataset
dataset_name = "lcel-teacher-eval"

In [20]:
# Run base case
try:
    experiment_results_ = evaluate(
        predict_base_case,
        data=dataset_name,
        evaluators=code_evalulator,
        experiment_prefix=f"test-without-langgraph-{llm}",
        max_concurrency=2,
        metadata={
            "llm": llm,
        },
    )
except:
    print("Please setup LangSmith")

/home/yeonwoo/miniforge3/envs/langgraph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'test-without-langgraph-client=<openai.resources.chat.completions.Completions object at 0x7fd0637f43d0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fd06318f190> root_client=<openai.OpenAI object at 0x7fd06331d9d0> root_async_client=<openai.AsyncOpenAI object at 0x7fd063142690> model_name='gpt-4o-2024-08-06' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') default_headers={'anthropic-beta': 'tools-2024-04-04'}-4ca32873' at:
https://smith.langchain.com/o/0708e8be-8469-58eb-8179-f15e52e09b25/datasets/ea83dffd-1b68-441c-a6c1-62f28b7b09a0/compare?selectedSessions=ccda3473-53ce-4dbd-abf2-5d3ffbf7adf0




11it [00:25,  2.42s/it]

Using itemgetter: What is LCEL?
Using lambda: What is LCEL?
Using get method: What is LCEL?


20it [00:45,  2.25s/it]


In [21]:
# Run with langgraph
try:
    experiment_results = evaluate(
        predict_langgraph,
        data=dataset_name,
        evaluators=code_evalulator,
        experiment_prefix=f"test-with-langgraph-{llm}-{flag}",
        max_concurrency=2,
        metadata={
            "llm": llm,
            "feedback": flag,
        },
    )
except:
    print("Please setup LangSmith")

View the evaluation results for experiment: 'test-with-langgraph-client=<openai.resources.chat.completions.Completions object at 0x7fd0637f43d0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fd06318f190> root_client=<openai.OpenAI object at 0x7fd06331d9d0> root_async_client=<openai.AsyncOpenAI object at 0x7fd063142690> model_name='gpt-4o-2024-08-06' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') default_headers={'anthropic-beta': 'tools-2024-04-04'}-do not reflect-d3dda7e8' at:
https://smith.langchain.com/o/0708e8be-8469-58eb-8179-f15e52e09b25/datasets/ea83dffd-1b68-441c-a6c1-62f28b7b09a0/compare?selectedSessions=f0ed8bc4-5ad7-4eb7-9e6f-2336142a1deb




0it [00:00, ?it/s]

---GENERATING CODE SOLUTION------GENERATING CODE SOLUTION---

---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
{'a': 1, 'b': 2, 'c': 3}
---NO CODE TEST FAILURES---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
{'a': 1, 'b': 2, 'c': 3}
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---


1it [00:12, 12.91s/it]

---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---


...

---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUT

<string>:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
<string>:13: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
<string>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
<string>:20: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.




Hello there! How can I assist you?
---NO CODE TEST FAILURES---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---


13it [01:54,  8.77s/it]

---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---
Please setup LangSmith
